# Training Analysis Project For Code Louisville Summer 2020 Program

## Import pandas and CSV file
Import pandas and read the Samplecomments2 CSV file, storing it in df

In [5]:
import pandas as pd

In [10]:
training_df = pd.read_csv('Samplecomments2.csv')
restaurant_df = pd.read_csv('Restaurantdata.csv')

## Take a look at the data in the CSV Files
Use: 
- .head to see the data in a spreadsheet format (put 10 in the parentheses to see all 10 rows)
- .shape to see how many rows/columns there are

Determine where there's a matching individual identifier to combine both DataFrames.

In [7]:
training_df.head(10)

,Username,Date,Restaurant,OSC,Receiving,Wish,Feedback
0,ABC123,5/20/2020,B000001,4.00,Yes,Shorter; More shoulder-to-shoulder,I love the training!
1,ABC234,5/20/2020,B000002,3.20,No,More shoulder-to-shoulder,The training is alright.
2,ABC345,5/20/2020,B000003,3.80,Yes,More computer,Training is okay.
3,ABC456,5/20/2020,B000004,2.80,No,Longer,Training is terrible.
4,ABC567,5/19/2020,B000005,2.50,No,Shorter; More shoulder-to-shoulder,I hate the training.
5,ABC678,5/19/2020,B000006,3.10,No,Shorter; More computer,I wish I didn't have to take the training.
6,ABC789,5/19/2020,B000007,3.60,Yes,Shorter,The training is pretty good.
7,ABC890,5/15/2020,B000008,3.80,Yes,More shoulder-to-shoulder,The training is really helpful.
8,ABC901,5/15/2020,B000009,3.95,Yes,Longer; More shoulder-to-shoulder,Training helps my team and I perform our jobs ...
9,ABC902,5/14/2020,B000010,2.50,No,More computer,Training is a waste of time and I don't want i...


In [8]:
training_df.shape

(10, 7)

In [15]:
restaurant_df.head()

,Restaurant,SalesMil,Safety,WindowSeconds,BestofBestAward,FullyTrained,Staff,FollowSchedule
0,B000001,$2.2,Yes,65,Yes,Yes,40,Yes
1,B000002,$2.1,Yes,80,No,Yes,36,Yes
2,B000003,$2.0,Yes,75,Yes,Yes,38,Yes
3,B000004,$1.6,Yes,120,No,Yes,17,No
4,B000005,$1.2,No,130,No,No,13,No


In [16]:
restaurant_df.shape

(20, 8)

## Combine DataFrames

Join the two DataFrames (training_df & restaurant_df) into a new one for analysis purposes. Restaurant_df contains more data, but we only care about looking at restaurants that completed the training survey (training_df), so we will want to do a left join.

In [18]:
complete_df = pd.merge(left=training_df, right=restaurant_df, how='left', left_on='Restaurant', right_on='Restaurant')

In [21]:
complete_df.head(10)

,Username,Date,Restaurant,OSC,Receiving,Wish,Feedback,SalesMil,Safety,WindowSeconds,BestofBestAward,FullyTrained,Staff,FollowSchedule
0,ABC123,5/20/2020,B000001,4.00,Yes,Shorter; More shoulder-to-shoulder,I love the training!,$2.2,Yes,65,Yes,Yes,40,Yes
1,ABC234,5/20/2020,B000002,3.20,No,More shoulder-to-shoulder,The training is alright.,$2.1,Yes,80,No,Yes,36,Yes
2,ABC345,5/20/2020,B000003,3.80,Yes,More computer,Training is okay.,$2.0,Yes,75,Yes,Yes,38,Yes
3,ABC456,5/20/2020,B000004,2.80,No,Longer,Training is terrible.,$1.6,Yes,120,No,Yes,17,No
4,ABC567,5/19/2020,B000005,2.50,No,Shorter; More shoulder-to-shoulder,I hate the training.,$1.2,No,130,No,No,13,No
5,ABC678,5/19/2020,B000006,3.10,No,Shorter; More computer,I wish I didn't have to take the training.,$1.1,Yes,100,No,Yes,18,No
6,ABC789,5/19/2020,B000007,3.60,Yes,Shorter,The training is pretty good.,$1.9,Yes,90,No,Yes,19,No
7,ABC890,5/15/2020,B000008,3.80,Yes,More shoulder-to-shoulder,The training is really helpful.,$1.8,Yes,70,No,Yes,22,Yes
8,ABC901,5/15/2020,B000009,3.95,Yes,Longer; More shoulder-to-shoulder,Training helps my team and I perform our jobs ...,$1.8,Yes,75,Yes,Yes,37,Yes
9,ABC902,5/14/2020,B000010,2.50,No,More computer,Training is a waste of time and I don't want i...,$1.1,No,180,No,No,17,No


In [20]:
complete_df.shape

(10, 14)

## Look at how people responded to the "Receiving" question
Take a look at how many people said they are receiving what they need to train their people on promotions versus the number of people who do not ("normalize" the counts to represent as a percentage for both yes/no).

In [23]:
complete_df['Receiving'].value_counts(normalize=True)

Yes    0.5
No     0.5
Name: Receiving, dtype: float64

## Examine the OSC scores compared to the "Receiving" response
Print:
- Average OSC score of those who said no vs. yes
- Median OSC score of no vs.  yes

## Look at responses to the "Wish" question
Keeping in mind that people can select multiple wishes, analyze:
- How many people selected shorter training
- Splitting up every entry and stack so each individual item can be counted; then, check the count for each